# Watch Me Code 3: Mapping with Folium

Folium is a Python wrapper library for the OpenStreetMaps api. It allows you to place data on a map in a variety of ways.


In [1]:
! pip install folium

You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import folium
import pandas as pd
import random 

In [3]:
# we need to center the map in the middle of the US. I googled for the location.
CENTER_US = (39.8333333,-98.585522)
london = (51.5074, -0.1278)
map = folium.Map(location=CENTER_US, zoom_start=2)
map

## Map Pins

In [7]:
# read in a data file of IP address to locations.
data = pd.read_csv('https://raw.githubusercontent.com/mafudge/datasets/master/clickstream/ip_lookup.csv')
data.sample(5)

,IP,Country,State,City,ApproxLat,ApproxLng
10,56.216.127.219,USA,NC,Raleigh,35.779590,-78.638179
15,8.37.70.112,USA,CA,Los Angeles,34.052234,-118.243685
17,8.37.70.99,USA,CA,Los Angeles,34.052234,-118.243685
13,8.37.70.170,USA,CA,Los Angeles,34.052234,-118.243685
16,8.37.70.226,USA,CA,Los Angeles,34.052234,-118.243685


In [8]:
# Let's place each location on the map
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (row['City'],row['State'])
                          )
    map.add_child(marker)
map

In [14]:
# Same thing with a different icon and colors. Icons come from http://fontawesome.io/icons/ but its an older version.
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
            'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
            'darkpurple', 'pink', 'lightblue', 'lightgreen',
            'gray', 'black', 'lightgray']
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (row['City'],row['State']),
                    icon = folium.Icon(color = random.choice(colors), icon='user')
                          )
    map.add_child(marker)
map

In [6]:
# There are other map tiles available. See https://folium.readthedocs.io/en/latest/quickstart.html
# Instead of Markers we use circles colors are HTML color codes http://htmlcolorcodes.com/
CENTER_US = (39.8333333,-98.585522)
map2 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Stamen Toner')
for row in data.to_records():
    map2.add_children(folium.CircleMarker(location=(row['ApproxLat'],row['ApproxLng']), 
                    popup=row['City'], radius=10000, color='#0000FF',  fill_color='#FF3333'))
    
map2

NameError: name 'data' is not defined

# Choropleths

Choropleths are cartographic overlays based on boundries defined in a geo JSON file.


In [13]:
# State level geo-json overlay choropleth
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map3.choropleth(geo_path=state_geojson)
map3


TypeError: choropleth() got an unexpected keyword argument 'geo_path'

In [11]:
states = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')
state_counts = pd.DataFrame( {'Counts' : data['State']. value_counts() } ).sort_index()
state_counts['StateCode'] = state_counts.index
state_data = states.merge(state_counts, how="left", left_on='Abbreviation', right_on='StateCode')
state_data = state_data[['Abbreviation','Counts']]
state_data = state_data.fillna(0)
#state_data


In [12]:
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map3.choropleth(geo_path=state_geojson,data=state_data, columns=['Abbreviation','Counts'], 
                key_on ='feature.id', fill_color='BuGn', legend_name='Website Visitors')
map3

TypeError: choropleth() got an unexpected keyword argument 'geo_path'

In [17]:
# Here's a more straigtforward example with unemployment data:
unemployment = pd.read_csv('https://raw.githubusercontent.com/wrobstory/vincent/master/examples/data/US_Unemployment_Oct2012.csv')
state_geojson = 'WMC3-us-states.json'
map4 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map4.choropleth(geo_path=state_geojson,data=unemployment, 
                columns=['State','Unemployment'], key_on ='feature.id', fill_color='YlOrRd', legend_name='2012 US Unemployment')
map4


TypeError: choropleth() got an unexpected keyword argument 'geo_path'